In [3]:
# importing libraries 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import seaborn as sb
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import keras.utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.utils.fixes import loguniform

In [6]:
# importing the Elastic Modulus as the output 
# Data are imported from the 3-point bending experiment 

E=np.array ([57.71,
20.24,
32.43,
22.63,
64.28,
17.95,
17.67,
39.44,
9.33,
40.68,
26.56,
32.69,
57.93,
67.88,
222.46,
])

# repeating the E for the 8 different augmented images (rotated, mirrored, etc.) as they are having the same E 
y=np.repeat(E, 8)

In [7]:
y

array([ 57.71,  57.71,  57.71,  57.71,  57.71,  57.71,  57.71,  57.71,
        20.24,  20.24,  20.24,  20.24,  20.24,  20.24,  20.24,  20.24,
        32.43,  32.43,  32.43,  32.43,  32.43,  32.43,  32.43,  32.43,
        22.63,  22.63,  22.63,  22.63,  22.63,  22.63,  22.63,  22.63,
        64.28,  64.28,  64.28,  64.28,  64.28,  64.28,  64.28,  64.28,
        17.95,  17.95,  17.95,  17.95,  17.95,  17.95,  17.95,  17.95,
        17.67,  17.67,  17.67,  17.67,  17.67,  17.67,  17.67,  17.67,
        39.44,  39.44,  39.44,  39.44,  39.44,  39.44,  39.44,  39.44,
         9.33,   9.33,   9.33,   9.33,   9.33,   9.33,   9.33,   9.33,
        40.68,  40.68,  40.68,  40.68,  40.68,  40.68,  40.68,  40.68,
        26.56,  26.56,  26.56,  26.56,  26.56,  26.56,  26.56,  26.56,
        32.69,  32.69,  32.69,  32.69,  32.69,  32.69,  32.69,  32.69,
        57.93,  57.93,  57.93,  57.93,  57.93,  57.93,  57.93,  57.93,
        67.88,  67.88,  67.88,  67.88,  67.88,  67.88,  67.88,  67.88,
      

In [ ]:
import glob
import os
from PIL import Image,ImageOps

my_path = "Imageset_combined/"
# Location to move images to

# Get List of all images
files = glob.glob(my_path + '/**/*.png', recursive=True)
images_dataset = []
filename_list=[]
# For each image
for file in files:
    # Get File name and extension
    filename = os.path.basename(file)
    
    # Open each image 
    im=Image.open(file).convert('L')
    filename_list.append(filename)
    img = np.array(im)
    img = img.flatten() 
    images_dataset.append(img) 
    
X=np.array(images_dataset)


In [ ]:
y.shape

(120,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n_components = 96  # number of X_train samples ( PCA is limited to the minimum number of Samples and Features) 

print("Extracting the top %d eigenvalues from %d images" % (n_components, X_train.shape[0]))

pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)


Extracting the top 96 eigenvalues from 96 images


In [ ]:
X_train_pca.shape

(96, 96)

#### Building the model with PCA

In [ ]:
model_with_pca=tf.keras.models.Sequential()  # initiazling 
model_with_pca.add(tf.keras.layers.Dense(96,activation='relu'))  # first hidden layer with 96 neurons = 96 features (PCA)
model_with_pca.add(tf.keras.layers.Dense(48,activation='relu'))  # second hidden layer with 48 neurons 
model_with_pca.add(tf.keras.layers.Dense(24,activation='relu'))  # third hidden layer with 24 neurons 
model_with_pca.add(tf.keras.layers.Dense(1)) # output layer 
model_with_pca.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
#model.fit(X_train, y_train, batch_size = 8, epochs = 100)
History = model_with_pca.fit(X_train_pca, y_train, validation_split=0.2, batch_size = 8, epochs = 50, verbose=0)

In [ ]:
model_with_pca.evaluate(X_test_pca,y_test)

1/1 [==============================] - 0s 28ms/step - loss: 136893650212626142787606151168.0000 - accuracy: 0.0000e+00


[1.3689365021262614e+29, 0.0]

#### Building the model without PCA

In [ ]:
model_without_pca=tf.keras.models.Sequential()  # initiazling 
model_without_pca.add(tf.keras.layers.Dense(96,activation='relu'))  # first hidden layer with 96 neurons = 96 features (PCA)
model_without_pca.add(tf.keras.layers.Dense(48,activation='relu'))  # second hidden layer with 48 neurons 
model_without_pca.add(tf.keras.layers.Dense(24,activation='relu'))  # third hidden layer with 24 neurons 
model_without_pca.add(tf.keras.layers.Dense(1)) # output layer 
model_without_pca.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
#model.fit(X_train, y_train, batch_size = 8, epochs = 100)
History = model_without_pca.fit(X_train, y_train, validation_split=0.2, batch_size = 8, epochs = 50, verbose=0)



KeyboardInterrupt



In [ ]:
model_without_pca.evaluate(X_test,y_test)